In [76]:
# Библиотеки и плюшки
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split,KFold, cross_val_score,GridSearchCV
from sklearn.linear_model import RidgeCV,LassoCV
from sklearn import linear_model
from sklearn.metrics import r2_score,f1_score,accuracy_score
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder,MinMaxScaler,PolynomialFeatures
from sklearn.compose import make_column_transformer,ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC,LinearSVC

import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Переменные 
RANDOM_STATE = 42

# Загрузка данных
dataset = load_boston()

X = pd.DataFrame(dataset.data)
X.columns = dataset.feature_names
y = dataset.target

* CRIM - уровень преступности на душу населения
* ZN - доля земель под жилую застройку зонирована под участки площадью более 25 000 кв. футов.
* INDUS - доля акров неторгового бизнеса на город.
* CHAS - 1, если граничит с рекой, 0 иначе
* NOX - концентрация оксидов азота (частей на 10 миллионов)
* RM - среднее количество комнат в квартире
* AGE - доля жилых домов, построенных до 1940 г.
* DIS - взвешенные расстояния до пяти центров занятости Бостона
* RAD - индекс доступности к радиальным магистралям
* TAX - полная ставка налога на имущество за 10 000 долларов США
* PTRATIO - соотношение учеников и учителей по городам
* B - 1000(Bk - 0.63)^2, где Bk доля чернокожих людей по городам
* LSTAT - % более низкого статуса населения
* MEDV - Средняя стоимость домов в 1000 долларов.

1. Разделите выборку на обучающую и тестовую в отношении 80%/20%

In [2]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    train_size=0.2, 
                                                    random_state=RANDOM_STATE)

2. Обучите стандартную регрессию, а также Ridge и  Lasso и параметрами по умолчанию и выведите их R2 на тестовой выборке

In [3]:
linearRegression = linear_model.LinearRegression().fit(X_train, y_train)
ridgeRegression = linear_model.Ridge().fit(X_train, y_train)
lassoRegression = linear_model.Lasso().fit(X_train, y_train)

y_pred_LiR = linearRegression.predict(X_test)
y_pred_RR = ridgeRegression.predict(X_test)
y_pred_LR = lassoRegression.predict(X_test)

linearR_r2_score = r2_score(y_test,y_pred_LiR)
ridgeR_r2_score = r2_score(y_test,y_pred_RR)
lassoR_r2_score = r2_score(y_test,y_pred_LR)

print("linearRegression R2: {}".format(linearR_r2_score))
print("ridgeRegression R2: {}".format(ridgeR_r2_score))
print("lassoRegression R2: {}".format(lassoR_r2_score))

linearRegression R2: 0.7079608143498461
ridgeRegression R2: 0.7050279992803692
lassoRegression R2: 0.6535653753160682


3. Для Ridge и Lasso подберите коэффициент регуляризации(используйте GridSearchCV, RidgeCV, LassoCV) в пределах от $10^{-5}$ до $10^5$ (по степеням 10). Посчитайте R2 на тестовой выборке по лучшим моделям и сравните с предыдущими результатами. Напишите как изменился результат

In [4]:
data_alphas = [1.e-5, 1.e-4, 1.e-3, 1.e-2, 1.e-1, 1.e+0, 1.e+1,
      1.e+2, 1.e+3, 1.e+4, 1.e+5]

ridgeGSScore = GridSearchCV(ridgeRegression, { 'alpha': data_alphas }, cv=5,n_jobs = -1).fit(X_train, y_train)
lassoGSScore = GridSearchCV(lassoRegression, { 'alpha': data_alphas }, cv=5,n_jobs = -1).fit(X_train, y_train)

ridgeRegression = linear_model.Ridge(alpha=ridgeGSScore.best_params_['alpha']).fit(X_train, y_train)
lassoRegression = linear_model.Lasso(alpha=lassoGSScore.best_params_['alpha']).fit(X_train, y_train)

y_pred_ridge_best = ridgeRegression.predict(X_test)
y_pred_lasso_best = lassoRegression.predict(X_test)

ridgeR_r2_score_best = r2_score(y_test,y_pred_ridge_best)
lassoR_r2_score_best = r2_score(y_test,y_pred_lasso_best)

print("ridgeRegression best Alpha R2: {}".format(ridgeR_r2_score_best))
print("lassoRegression best Alpha R2: {}".format(lassoR_r2_score_best))

print('R2 Ridge изменилось на: %f' % (ridgeR_r2_score_best - ridgeR_r2_score))
print('R2 Lasso изменилось на: %f' % (lassoR_r2_score_best - lassoR_r2_score))

# Результат лучше

ridgeRegression best Alpha R2: 0.7065995399152001
lassoRegression best Alpha R2: 0.7049260440433983
R2 Ridge изменилось на: 0.001572
R2 Lasso изменилось на: 0.051361


4. Проведите масштабирование выборки(используйте Pipeline, StandardScaler, MinMaxScaler), посчитайте R2 и сравните с предыдущими результатами. Напишите как изменился результат

In [5]:
pipeRidge = Pipeline(
    [
        ('SS', StandardScaler()),
        ('MMS', MinMaxScaler()),
        ('R',linear_model.Ridge(alpha=ridgeGSScore.best_params_['alpha']))
    ]
).fit(X_train, y_train)
pipeLasso = Pipeline(
    [
        ('SS', StandardScaler()),
        ('MMS', MinMaxScaler()),
        ('L',linear_model.Lasso(alpha=lassoGSScore.best_params_['alpha']))
    ]
).fit(X_train, y_train)

y_pred_pipeRidge = pipeRidge.predict(X_test)
y_pred_pipeLasso = pipeLasso.predict(X_test)

r2_score_pipeRidge = r2_score(y_test,y_pred_pipeRidge)
r2_score_pipeLasso = r2_score(y_test,y_pred_pipeLasso)

print("ridgeRegression Unif Alpha R2: {}".format(r2_score_pipeRidge))
print("lassoRegression Unif Alpha R2: {}".format(r2_score_pipeLasso))

print('R2 Ridge изменилось на: %f' % (r2_score_pipeRidge - ridgeR_r2_score_best))
print('R2 Lasso изменилось на: %f' % (r2_score_pipeLasso - lassoR_r2_score_best))

# Результат хуже для Ridge и для Lasso

ridgeRegression Unif Alpha R2: 0.5226581021852552
lassoRegression Unif Alpha R2: 0.6808498310031998
R2 Ridge изменилось на: -0.183941
R2 Lasso изменилось на: -0.024076


5. Подберите коэффициент регуляризации для Ridge и Lasso на масштабированных данных, посчитайте R2 и сравните с предыдущими результатами. Напишите как изменился результат

In [6]:
pipeRidgeGSScore = GridSearchCV(pipeRidge, { 'R__alpha': data_alphas }, cv=5,n_jobs = -1).fit(X_train, y_train)
pipeLassoGSScore = GridSearchCV(pipeLasso, { 'L__alpha': data_alphas }, cv=5,n_jobs = -1).fit(X_train, y_train)

ridgeRegression = linear_model.Ridge(alpha=pipeRidgeGSScore.best_params_['R__alpha']).fit(X_train, y_train)
lassoRegression = linear_model.Lasso(alpha=pipeLassoGSScore.best_params_['L__alpha']).fit(X_train, y_train)

y_pred_ridge_pipe = ridgeRegression.predict(X_test)
y_pred_lasso_pipe = lassoRegression.predict(X_test)

ridgeR_r2_score_pipe = r2_score(y_test,y_pred_ridge_pipe)
lassoR_r2_score_pipe = r2_score(y_test,y_pred_lasso_pipe)

print("ridgeRegression Pipe Alpha R2: {}".format(ridgeR_r2_score_pipe))
print("lassoRegression Pipe Alpha R2: {}".format(lassoR_r2_score_pipe))

print('R2 Ridge изменилось на: %f' % (ridgeR_r2_score_pipe - r2_score_pipeRidge ))
print('R2 Lasso изменилось на: %f' % (lassoR_r2_score_pipe - r2_score_pipeLasso ))

# Результат лучше, такой же как и без масштабирования 

ridgeRegression Pipe Alpha R2: 0.7050279992803692
lassoRegression Pipe Alpha R2: 0.7049260440433983
R2 Ridge изменилось на: 0.182370
R2 Lasso изменилось на: 0.024076


6. Добавьте попарные произведения признаков и их квадраты (используйте PolynomialFeatures) на масштабированных признаках, посчитайте R2 и сравните с предыдущими результатами. Напишите как изменился результат

In [7]:
pipeRidgePoly = Pipeline(
    [
        ('SS', StandardScaler()),
        ('MMS', MinMaxScaler()),
        ('Poly',PolynomialFeatures(degree=2)),
        ('R',linear_model.Ridge(alpha=ridgeGSScore.best_params_['alpha']))
    ]
).fit(X_train, y_train)
pipeLassoPoly = Pipeline(
    [
        ('SS', StandardScaler()),
        ('MMS', MinMaxScaler()),
        ('Poly',PolynomialFeatures(degree=2)),
        ('L',linear_model.Lasso(alpha=lassoGSScore.best_params_['alpha']))
    ]
).fit(X_train, y_train)

y_pred_pipeRidgePoly = pipeRidgePoly.predict(X_test)
y_pred_pipeLassoPoly = pipeLassoPoly.predict(X_test)

r2_score_pipeRidgePoly = r2_score(y_test,y_pred_pipeRidgePoly)
r2_score_pipeLassoPoly = r2_score(y_test,y_pred_pipeLassoPoly)

print("ridgeRegression Poly Alpha R2: {}".format(r2_score_pipeRidgePoly))
print("lassoRegression Poly Alpha R2: {}".format(r2_score_pipeLassoPoly))

print('R2 Ridge изменилось на: %f' % (r2_score_pipeRidgePoly - ridgeR_r2_score_pipe))
print('R2 Lasso изменилось на: %f' % (r2_score_pipeLassoPoly - lassoR_r2_score_pipe))

# Результат хуже для Ridge, лучше для Lasso

ridgeRegression Poly Alpha R2: 0.6624219116203638
lassoRegression Poly Alpha R2: 0.7374909201262771
R2 Ridge изменилось на: -0.042606
R2 Lasso изменилось на: 0.032565


7. Подберите наилучшую модель (используйте Pipeline, GridSearchSCV) подбирая тип регуляризации (L1,L2), коэффициент регуляризации, метод масштабирования и степень полинома в PolynomialFeatures. Выведите итоговые параметры и результат R2. Напишите как изменился R2 по сравнению с предыдущими экспериментами

In [ ]:
pipeUltra = Pipeline(
    [
        ('Scaler', 'passthrough'),
        ('Poly', 'passthrough'),
        ('Classif', 'passthrough')
    ]
)

params_grid = [
    {
    'Scaler': [None, StandardScaler(), MinMaxScaler()],
    'Poly': [PolynomialFeatures(degree=2),PolynomialFeatures(degree=4)],
    'Classif': [linear_model.Ridge(), linear_model.Lasso()],
    'Classif__alpha': data_alphas,
    'Classif__tol': [0.001, 0.01, 0.1, 1.0]
    }
]

gridSearchPipeUltra = GridSearchCV(pipeUltra, params_grid, scoring='r2', cv=3, n_jobs=-1).fit(X_train, y_train)

In [9]:
gridSearchPipeUltra.best_params_

{'Classif': Lasso(alpha=0.1, tol=1.0),
 'Classif__alpha': 0.1,
 'Classif__tol': 1.0,
 'Poly': PolynomialFeatures(degree=4),
 'Scaler': StandardScaler()}

In [10]:
pipeLassoPolyUltra = Pipeline(
    [
        ('SS',  StandardScaler()),
        ('Poly',PolynomialFeatures(degree=4)),
        ('L',   linear_model.Lasso(alpha=0.1, tol=1.0))
    ]
).fit(X_train, y_train)

y_pred_pipeLassoUltra = pipeLassoPolyUltra.predict(X_test)

r2_score_pipeLassoUltra = r2_score(y_test,y_pred_pipeLassoUltra)

print("lassoRegression Poly Alpha R2: {}".format(r2_score_pipeLassoUltra))

print('R2 Lasso изменилось на: %f' % (r2_score_pipeLassoUltra - r2_score_pipeLassoPoly))

# Немного ухудшился :) 

lassoRegression Poly Alpha R2: 0.6670508079682187
R2 Lasso изменилось на: -0.070440


# -----------------------------------------------------------------------------------------------------------------------

http://archive.ics.uci.edu/ml/datasets/Adult

In [11]:
link = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/adult-all.csv'
df = pd.read_csv(link, header=None)

df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,39,Private,215419,Bachelors,13,Divorced,Prof-specialty,Not-in-family,White,Female,0,0,36,United-States,<=50K
48838,64,?,321403,HS-grad,9,Widowed,?,Other-relative,Black,Male,0,0,40,United-States,<=50K
48839,38,Private,374983,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,50,United-States,<=50K
48840,44,Private,83891,Bachelors,13,Divorced,Adm-clerical,Own-child,Asian-Pac-Islander,Male,5455,0,40,United-States,<=50K


8. Разделите выборку на признаки и целевую переменную(колонка со зачениями {<=50K,>50K}). Замените целевую переменную на числовые значения.

In [12]:
X = df.drop(14, 1) 
y = df[14] 

label = LabelEncoder()
dicts = {}
label.fit(y.drop_duplicates()) #задаем список значений для кодирования
dicts['label'] = list(label.classes_)
y = label.transform(y) #заменяем значения из списка кодами закодированных элементов 

dicts
X
y

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    train_size=0.2, 
                                                    random_state=RANDOM_STATE)

LabelEncoder()

{'label': ['<=50K', '>50K']}

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,39,Private,215419,Bachelors,13,Divorced,Prof-specialty,Not-in-family,White,Female,0,0,36,United-States
48838,64,?,321403,HS-grad,9,Widowed,?,Other-relative,Black,Male,0,0,40,United-States
48839,38,Private,374983,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,50,United-States
48840,44,Private,83891,Bachelors,13,Divorced,Adm-clerical,Own-child,Asian-Pac-Islander,Male,5455,0,40,United-States


array([0, 0, 0, ..., 0, 0, 1])

9. Выясните, присутствуют ли в данных пропуски. Заполните их самыми частыми значениями (испольуйте SimpleImputer)

In [13]:
df.isna().sum()

# Пропусков нет

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
dtype: int64

In [14]:
# на Всякий пожарный

#imp_mean = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
#X = imp_mean.fit_transform(X)

10. Выберите колонки с числовыми и категориальными переменными.

In [15]:
numeric_cols = X.select_dtypes('number').columns.to_list()
categorical_cols = X.select_dtypes('object').columns.to_list()

numeric_cols
categorical_cols

[0, 2, 4, 10, 11, 12]

[1, 3, 5, 6, 7, 8, 9, 13]

11. Создайте пайплайн по обработке колонок(используйте OneHotEncoder,MinMaxScaler).

In [ ]:
preprocessor = ColumnTransformer(
    transformers=
    [
        ("cat", LabelEncoder(), categorical_cols)
    ]
)

model = Pipeline(
    [
        ("preprocessor", preprocessor)
    ]
)#.fit_transform(X_train)

# Ошибка при трансформации
#fit_transform() takes 2 positional arguments but 3 were given
#Откатил версию до skLearn 0.18, не понятно в чем беда :(

# Пришлось делать руками

for i in categorical_cols:
    label = LabelEncoder()
    ##dicts = {}
    label.fit(X[i].drop_duplicates()) #задаем список значений для кодирования
    ##dicts['label'] = list(label.classes_)
    X[i] = label.transform(X[i]) #заменяем значения из списка кодами закодированных элементов 

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    train_size=0.2, 
                                                    random_state=RANDOM_STATE)

In [17]:
X_train

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
39176,56,4,94156,11,9,2,1,0,4,1,0,0,50,39
47260,45,4,142909,11,9,2,3,0,4,1,0,0,40,39
17290,31,4,132601,11,9,4,8,3,4,1,0,0,40,39
45116,47,6,177533,15,10,2,3,0,4,1,0,0,40,39
760,36,4,101460,11,9,4,8,1,4,0,0,0,18,39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11284,35,4,200117,9,13,2,4,0,1,1,0,1887,50,0
44732,21,4,90896,11,9,4,7,3,4,0,0,0,40,39
38158,23,4,370057,11,9,0,1,1,4,0,0,0,40,39
860,18,4,216284,1,7,4,1,3,4,0,0,0,20,39


12. Посчитайте метрики accuracy и f1_score на предсказании только самого частого класса в целевой переменной.

In [18]:
counts = np.bincount(y)
freqclass = np.argmax(counts)

print("Самый частый класс : {}".format(freqclass))

freqArr = np.array([freqclass for _ in range(len(y))])

print("accuracy: {}".format(accuracy_score(y,freqArr)))
print("f1_score: {}".format(f1_score(y,freqArr)))

Самый частый класс : 0
accuracy: 0.7607182343065395
f1_score: 0.0


13. Посчитайте cross_val_score по алгоритмам LogisticRegression, SVC, LinearSVC по метрикам accuracy и f1_score.
Напишите удалось ли превзойти предыдущий результат.

In [ ]:
clf_LR   = linear_model.LogisticRegression().fit(X_train, y_train)
clf_SVC  = SVC().fit(X_train, y_train)
clf_LSVC = LinearSVC().fit(X_train, y_train)

cvs_LR_acc   = cross_val_score(clf_LR,   X, y, n_jobs =-1, scoring = 'accuracy').mean()
cvs_SVC_acc  = cross_val_score(clf_SVC,  X, y, n_jobs =-1, scoring = 'accuracy').mean()
cvs_LSVC_acc = cross_val_score(clf_LSVC, X, y, n_jobs =-1, scoring = 'accuracy').mean()

cvs_LR_f1    = cross_val_score(clf_LR,   X, y, n_jobs =-1, scoring = 'f1').mean()
cvs_SVC_f1   = cross_val_score(clf_SVC,  X, y, n_jobs =-1, scoring = 'f1').mean()
cvs_LSVC_f1  = cross_val_score(clf_LSVC, X, y, n_jobs =-1, scoring = 'f1').mean()

In [20]:
print('Точность LR  : %f' % (cvs_LR_acc))
print('Точность SVC : %f' % (cvs_SVC_acc))
print('Точность LSVC: %f' % (cvs_LSVC_acc))

print('F1 мера LR  : %f' % (cvs_LR_f1))
print('F1 мера SVC : %f' % (cvs_SVC_f1))
print('F1 мера LSVC: %f' % (cvs_LSVC_f1))

Точность LR  : 0.794112
Точность SVC : 0.798288
Точность LSVC: 0.782564
F1 мера LR  : 0.387847
F1 мера SVC : 0.279702
F1 мера LSVC: 0.367393


14. Можно заметить что в данных присутствуют значения '?', замените их самыми частыми значениями (испольуйте SimpleImputer)

In [ ]:
imp_symbol = SimpleImputer(missing_values='?', strategy='most_frequent')
df_updated = imp_symbol.fit_transform(X)

df_updated
#  Ошибка 'X' and 'missing_values' types are expected to be both numerical. Got X.dtype=int64 and  type(missing_values)=<class 'str'>.
# Скорее всего, после обновления какого-либо из пакетов не дает передать символ как пропуск

15. Посчитайте cross_val_score на новых данных. Напишите удалось ли улучшить результат.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    train_size=0.2, 
                                                    random_state=RANDOM_STATE)

clf_LR   = linear_model.LogisticRegression().fit(X_train, y_train)
clf_SVC  = SVC().fit(X_train, y_train)
clf_LSVC = LinearSVC().fit(X_train, y_train)

cvs_LR_acc   = cross_val_score(clf_LR,   X, y, n_jobs =-1, scoring = 'accuracy').mean()
cvs_SVC_acc  = cross_val_score(clf_SVC,  X, y, n_jobs =-1, scoring = 'accuracy').mean()
cvs_LSVC_acc = cross_val_score(clf_LSVC, X, y, n_jobs =-1, scoring = 'accuracy').mean()

cvs_LR_f1    = cross_val_score(clf_LR,   X, y, n_jobs =-1, scoring = 'f1').mean()
cvs_SVC_f1   = cross_val_score(clf_SVC,  X, y, n_jobs =-1, scoring = 'f1').mean()
cvs_LSVC_f1  = cross_val_score(clf_LSVC, X, y, n_jobs =-1, scoring = 'f1').mean()

In [23]:
print('Точность LR   (updated): %f' % (cvs_LR_acc))
print('Точность SVC  (updated): %f' % (cvs_SVC_acc))
print('Точность LSVC (updated): %f' % (cvs_LSVC_acc))

print('F1 мера LR   (updated): %f' % (cvs_LR_f1))
print('F1 мера SVC  (updated): %f' % (cvs_SVC_f1))
print('F1 мера LSVC (updated): %f' % (cvs_LSVC_f1))

Точность LR   (updated): 0.794112
Точность SVC  (updated): 0.798288
Точность LSVC (updated): 0.692156
F1 мера LR   (updated): 0.387847
F1 мера SVC  (updated): 0.279702
F1 мера LSVC (updated): 0.343636


16. Посчитайте cross_val_score, если просто удалить значения '?'. Напишите как изменился результат

In [ ]:
data_new = df.replace('?', np.nan).dropna()

X = data_new.drop(14, 1) 
y = data_new[14] 

numeric_cols = X.select_dtypes('number').columns.to_list()
categorical_cols = X.select_dtypes('object').columns.to_list()
# Пришлось делать руками
for i in categorical_cols:
    label = LabelEncoder()
    ##dicts = {}
    label.fit(X[i].drop_duplicates()) #задаем список значений для кодирования
    ##dicts['label'] = list(label.classes_)
    X[i] = label.transform(X[i]) #заменяем значения из списка кодами закодированных элементов 

label = LabelEncoder().fit(y.drop_duplicates())
#dicts = {}
#dicts['label'] = list(label.classes_)
y = label.transform(y) #заменяем значения из списка кодами закодированных элементов 

    
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    train_size=0.2, 
                                                    random_state=RANDOM_STATE)

In [ ]:
clf_LR   = linear_model.LogisticRegression().fit(X_train, y_train)
clf_SVC  = SVC().fit(X_train, y_train)
clf_LSVC = LinearSVC().fit(X_train, y_train)

cvs_LR_acc   = cross_val_score(clf_LR,   X, y, n_jobs =-1, scoring = 'accuracy').mean()
cvs_SVC_acc  = cross_val_score(clf_SVC,  X, y, n_jobs =-1, scoring = 'accuracy').mean()
cvs_LSVC_acc = cross_val_score(clf_LSVC, X, y, n_jobs =-1, scoring = 'accuracy').mean()

cvs_LR_f1    = cross_val_score(clf_LR,   X, y, n_jobs =-1, scoring = 'f1').mean()
cvs_SVC_f1   = cross_val_score(clf_SVC,  X, y, n_jobs =-1, scoring = 'f1').mean()
cvs_LSVC_f1  = cross_val_score(clf_LSVC, X, y, n_jobs =-1, scoring = 'f1').mean()

In [44]:
print('Точность LR   (updated): %f' % (cvs_LR_acc))
print('Точность SVC  (updated): %f' % (cvs_SVC_acc))
print('Точность LSVC (updated): %f' % (cvs_LSVC_acc))

print('F1 мера LR   (updated): %f' % (cvs_LR_f1))
print('F1 мера SVC  (updated): %f' % (cvs_SVC_f1))
print('F1 мера LSVC (updated): %f' % (cvs_LSVC_f1))

# Параметры ухудшились

Точность LR   (updated): 0.785459
Точность SVC  (updated): 0.790832
Точность LSVC (updated): 0.676911
F1 мера LR   (updated): 0.401922
F1 мера SVC  (updated): 0.277760
F1 мера LSVC (updated): 0.295462


 17. Посчитайте cross_val_score для RandomForestClassifier,GradientBoostingClassifier. Напишите как изменился результат и какой вывод можно из этого сделать.

In [45]:
clf_RF   = RandomForestClassifier(n_estimators=100,max_depth=10, random_state=RANDOM_STATE).fit(X_train, y_train)
clf_GBC  = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=10, random_state=RANDOM_STATE).fit(X_train, y_train)

cvs_RF_acc   = cross_val_score(clf_RF,   X, y, n_jobs =-1, scoring = 'accuracy').mean()
cvs_GBC_acc  = cross_val_score(clf_GBC,  X, y, n_jobs =-1, scoring = 'accuracy').mean()

cvs_RF_f1    = cross_val_score(clf_RF,   X, y, n_jobs =-1, scoring = 'f1').mean()
cvs_GBC_f1   = cross_val_score(clf_GBC,  X, y, n_jobs =-1, scoring = 'f1').mean()

In [46]:
print('Точность RF  : %f' % (cvs_RF_acc))
print('Точность GBC : %f' % (cvs_GBC_acc))

print('F1 мера RF   : %f' % (cvs_RF_f1))
print('F1 мера GBC  : %f' % (cvs_GBC_f1))

# RandomForestClassifier,GradientBoostingClassifier показывают себя значительно лучше на указанных данных

Точность RF  : 0.854695
Точность GBC : 0.840719
F1 мера RF   : 0.652035
F1 мера GBC  : 0.666356


18. Подберите наилучшую модель, подбирая методы обработки колонок - масштабирование признаков, кодирование признаков и заполнение пропусков. Параметры алгоритмов оставьте по умолчанию. Выведите итоговые параметры и результат accuracy и f1_score.

In [ ]:
columnTransf_MO = ColumnTransformer([
    ('scale',  MinMaxScaler(), numeric_cols),
    ('enc', OneHotEncoder(),  categorical_cols)
])

columnTransf_SO = ColumnTransformer([
    ('scale',  StandardScaler(), numeric_cols),
    ('enc', OneHotEncoder(),  categorical_cols)
])

columnTransf_ML = ColumnTransformer([
    ('scale',  MinMaxScaler(), numeric_cols),
    ('enc', LabelEncoder(),  categorical_cols)
])

columnTransf_SL = ColumnTransformer([
    ('scale',  StandardScaler(), numeric_cols),
    ('enc', LabelEncoder(),  categorical_cols)
])

pipe = Pipeline(
    [
        #('fill_miss', SimpleImputer(missing_values='?', strategy='most_frequent')), # Все еще ошибка,
        # #  Ошибка 'X' and 'missing_values' types are expected to be both numerical. Got X.dtype=int64 and  type(missing_values)=<class 'str'>.
        # не могу её решить, пока что запущу без заполнения
        ('transform', 'passthrough'),
        ('classif',   'passthrough')
    ]
)

params = [
    {
        'transform': 
        [
            columnTransf_MO, 
            columnTransf_SO,
            columnTransf_ML,
            columnTransf_SL
        ],
        'classif': 
        [
            RandomForestClassifier(),
            GradientBoostingClassifier(),
            linear_model.LogisticRegression(),
            SVC(),
            LinearSVC()
        ]
    }
]

search = GridSearchCV(pipe, params, scoring=['accuracy', 'f1'], refit='f1', cv=3, n_jobs=-1).fit(X_train, y_train)

In [75]:
print(search.best_params_)

print('Точность : %f' % (np.nanmean(search.cv_results_['mean_test_accuracy']) ))
print('F1 мера  : %f' % (np.nanmean(search.cv_results_['mean_test_f1'])))

# Показатели GradientBoostingClassifier улучшились

0.6897928230413181
{'classif': GradientBoostingClassifier(), 'transform': ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                 [0, 2, 4, 10, 11, 12]),
                                ('enc', OneHotEncoder(),
                                 [1, 3, 5, 6, 7, 8, 9, 13])])}
Точность : 0.848363
F1 мера  : 0.669311
